# Question 2

- This problem is aimed at recommanding products by finding products that are frequently browsered together 

In [79]:
from pyspark import SparkConf, SparkContext
import re
from itertools import combinations, permutations

In [29]:
SUPPORT = 100

In [2]:
conf = SparkConf()
sc = SparkContext(conf=conf)
lines = sc.textFile("q2/data/browsing.txt")

In [47]:
baskets = lines.map(lambda line: \
                    re.split(r'[^\w]+', line))\
              .map(lambda line: [v for v in line[0:-1]])

In [114]:
freq_items_1st = baskets.map(lambda line: [(v,1) for v in line])\
                        .flatMap(lambda x: x)\
                        .reduceByKey(lambda n1,n2: n1+n2)\
                        .filter(lambda x: x[1]>=SUPPORT)

keys_1st = freq_items_1st.keys()\
                         .collect()
dict_1st = dict(freq_items_1st.collect())

In [84]:
def filter_frequent(line, keys):
    '''pass the frequent items'''
    l = []
    for v in line:
        if v in keys: l.append(v)
    return l

## 2d

In [124]:
freq_items_2nd = baskets.filter(lambda x: filter_frequent(x, keys_1st))\
                    .flatMap(lambda x: \
                             [(key, 1) for key in permutations(x, 2)])\
                    .reduceByKey(lambda n1,n2: n1+n2)\
                    .filter(lambda x: x[1] >= SUPPORT)\

keys_2nd = freq_items_2nd.keys().collect()

In [121]:
confidence_2nd = freq_items_2nd.map(lambda x: (x[0], x[1]/dict_1st[x[0][0]]))\
                               .sortBy(lambda x: (-x[1], x[0][0]))

In [123]:
confidence_2nd.take(5)

[(('DAI93865', 'FRO40251'), 1.0),
 (('GRO85051', 'FRO40251'), 0.999176276771005),
 (('GRO38636', 'FRO40251'), 0.9906542056074766),
 (('ELE12951', 'FRO40251'), 0.9905660377358491),
 (('DAI88079', 'FRO40251'), 0.9867256637168141)]

## 2e

In [ ]:
freq_items_3rd = baskets.filter(lambda x: filter_frequent(x, keys_1st))\
                    .flatMap(lambda x: \
                             [(key, 1) for key in permutations(x, 2)])\
                    .reduceByKey(lambda n1,n2: n1+n2)\
                    .filter(lambda x: x[1] >= SUPPORT)\